In [1]:
import pandas as pd
import numpy as np

In [2]:
import plotly.express as px

In [3]:
import plotly.graph_objects as go
from ipywidgets import widgets

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/lulidezan97/dashboard_demo/master/bases/femasc_ponderado.csv')

In [13]:
#df.head()

In [6]:
#df.pregunta.unique()

In [17]:
df_1 = df.loc[(df.pregunta=='home office')]
df_2 = df.loc[(df.pregunta=='opinion cuarentena')]
df_3 = df.loc[(df.pregunta=='opinion circulacion')]

In [8]:
#df_uno

In [9]:
#% totales
## con filtrado para: edad (que se pueda seleccionar mas de una) y genero 

In [10]:
#seleccion de 

In [11]:
edades_buttons = widgets.RadioButtons(
    description='Rango etario:',
    value='Todas las edades',
    options=['Entre 18 y 25 años','Entre 26 y 35 años','Entre 36 y 45 años', 
             'Entre 46 y 55 años', 'Entre 56 y 65 años','Más de 66 años', 'Todas las edades']
)
genero_buttons = widgets.Dropdown(
    value='Todos',
    options=['Hombre', 'Mujer', 'Todos'],
    description='Género:',
)

container = widgets.HBox(children=[edades_buttons, genero_buttons])

In [18]:
# Assign an empty figure widget

fig_1 = go.Bar(y=df_1.respuesta,x=df_1.porcentaje, orientation="h")
fig_2 = go.Bar(y=df_2.respuesta,x=df_2.porcentaje, orientation="h")
fig_3 = go.Bar(y=df_3.respuesta,x=df_3.porcentaje, orientation="h")

g_1 = go.FigureWidget(data=[fig_1],
                    layout=go.Layout(
                        title=dict(
                            text=f"Actividades durante la cuarentena"
                        )))
g_2 = go.FigureWidget(data=[fig_2],
                    layout=go.Layout(
                        title=dict(
                            text=f"Opinión sobre la cuarentena"
                        )))
g_3 = go.FigureWidget(data=[fig_3],
                    layout=go.Layout(
                        title=dict(
                            text=f"Opinión sobre apertura de la circulación entre provincias"
                        )))

In [15]:
def response(change):
    if edades_buttons.value!='Todas las edades' and genero_buttons.value!='Todos':
        df_1 = df.loc[(df.pregunta=='home office') & (df.edad==edades_buttons.value) & (df.genero==genero_buttons.value)]
        y1 = df_1['respuesta']
        x1 = df_1['porcentaje']
        df_2 = df.loc[(df.pregunta=='opinion cuarentena') & (df.edad==edades_buttons.value) & (df.genero==genero_buttons.value)]
        y2 = df_2['respuesta']
        x2 = df_2['porcentaje']
        df_3 = df.loc[(df.pregunta=='opinion circulacion') & (df.edad==edades_buttons.value) & (df.genero==genero_buttons.value)]
        y3 = df_3['respuesta']
        x3 = df_3['porcentaje']
    elif edades_buttons.value=='Todas las edades' and genero_buttons.value!='Todos':
        df_1 = df.loc[(df.pregunta=='home office') & (df.genero==genero_buttons.value)]
        y1 = df_1['respuesta']
        x1 = df_1['porcentaje']
        df_2 = df.loc[(df.pregunta=='opinion cuarentena') & (df.genero==genero_buttons.value)]
        y2 = df_2['respuesta']
        x2 = df_2['porcentaje']
        df_3 = df.loc[(df.pregunta=='opinion circulacion') & (df.genero==genero_buttons.value)]
        y3 = df_3['respuesta']
        x3 = df_3['porcentaje']      
    elif edades_buttons.value!='Todas las edades' and genero_buttons.value=='Todos':
        df_1 = df.loc[(df.pregunta=='home office') & (df.edad==edades_buttons.value)]
        y1 = df_1['respuesta']
        x1 = df_1['porcentaje']
        df_2 = df.loc[(df.pregunta=='opinion cuarentena') & (df.edad==edades_buttons.value)]
        y2 = df_2['respuesta']
        x2 = df_2['porcentaje']
        df_3 = df.loc[(df.pregunta=='opinion circulacion') & (df.edad==edades_buttons.value)]
        y3 = df_3['respuesta']
        x3 = df_3['porcentaje']  
    else:
        df_1 = df.loc[(df.pregunta=='home office')]
        y1 = df_1['respuesta']
        x1 = df_1['porcentaje']
        df_2 = df.loc[(df.pregunta=='opinion cuarentena')]
        y2 = df_2['respuesta']
        x2 = df_2['porcentaje']
        df_3 = df.loc[(df.pregunta=='opinion circulacion')]
        y3 = df_3['respuesta']
        x3 = df_3['porcentaje']                       
    with g_1.batch_update():
        g_1.data[0].x = x1
        g_1.data[0].y = y1
        g_1.layout.xaxis.title = ' '
        g_1.layout.yaxis.title = ''
    with g_2.batch_update():
        g_2.data[0].y = y2
        g_2.data[0].x = x2
        g_2.layout.xaxis.title = ' '
        g_2.layout.yaxis.title = ''
    with g_3.batch_update():
        g_3.data[0].x = x3
        g_3.data[0].y = y3
        g_3.layout.xaxis.title = ' '
        g_3.layout.yaxis.title = ''
edades_buttons.observe(response, names="value")
genero_buttons.observe(response, names="value")

In [16]:
widgets.VBox([container,
              g_1,
             g_2,
             g_3])